In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
from glob import glob
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from torch_geometric.data import Data
from torch_geometric.data import DataLoader

import pickle
import sys
import requests

from types import MethodType
import importlib
from scperturb import *
import anndata as ad

In [2]:
import sys
sys.path.append('/data1/lichen/code/single_cell_perturbation/code_other/GEARS')
import gears
import importlib
importlib.reload(gears)
importlib.reload(gears.gears)
importlib.reload(gears.inference)
importlib.reload(gears.utils)
importlib.reload(gears.inference)
# importlib.reload(gears.PertData)
# importlib.reload(gears.GEARS)


from gears import PertData, GEARS
from gears.inference import evaluate, compute_metrics, deeper_analysis, \
                  non_dropout_analysis
                  
from sklearn.metrics import r2_score
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

In [3]:
sys.path.append("/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert_Data/")

In [4]:
import v1
from v1.utils import *
from v1.dataloader import *

In [5]:
importlib.reload(v1)
importlib.reload(v1.utils)
importlib.reload(v1.dataloader)

<module 'v1.dataloader' from '/data1/lichen/code/single_cell_perturbation/scPerturb/Byte_Pert_Data/v1/dataloader.py'>

# 所有gene pert的数据metadata

In [6]:
gene_pert_dict = \
{'AdamsonWeissman2016_GSM2406675_10X001': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 0},
 'AdamsonWeissman2016_GSM2406677_10X005': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 1,
  'multi_sgRNA': 0},
 'AdamsonWeissman2016_GSM2406681_10X010': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 0},
#  'DatlingerBock2017': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 1,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 1},
 'DatlingerBock2021': {'perturbation_type': 'CRISPR-cas9',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
 'DixitRegev2016': {'perturbation_type': 'CRISPR-cas9',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
#  'FrangiehIzar2021_RNA': {'perturbation_type': 'CRISPR-cas9', # 唯一一个multi celltype，先留着
#   'perturbation_state': 1,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 1},
 
#  'LaraAstiasoHuntly2023_exvivo': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 3,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 1},
#  'LaraAstiasoHuntly2023_invivo': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 3,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 1},
#  'LaraAstiasoHuntly2023_leukemia': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 3,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 1},
#  'LiangWang2023': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 3,
#   'multi_perturbation': 1,
#   'multi_sgRNA': 1},
#  'McFarlandTsherniak2020': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 3,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 0},
 'NormanWeissman2019_filtered': {'perturbation_type': 'CRISPRa',
  'perturbation_state': 1,
  'multi_perturbation': 1,
  'multi_sgRNA': 0},
 'PapalexiSatija2021_eccite_RNA': {'perturbation_type': 'CRISPR-cas9',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
 'PapalexiSatija2021_eccite_arrayed_RNA': {'perturbation_type': 'CRISPR-cas9',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
 'ReplogleWeissman2022_K562_essential': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
#  'ReplogleWeissman2022_K562_gwps': {'perturbation_type': 'CRISPRi', # pert太多了，先不测试
#   'perturbation_state': 1,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 1},
 'ReplogleWeissman2022_rpe1': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
#  'SantinhaPlatt2023': {'perturbation_type': 'CRISPR-cas9',
#   'perturbation_state': 3,
#   'multi_perturbation': 0,
#   'multi_sgRNA': 0},
 'ShifrutMarson2018': {'perturbation_type': 'CRISPR-cas9',
  'perturbation_state': 1,
  'multi_perturbation': 1,
  'multi_sgRNA': 1},
 'SunshineHein2023': {'perturbation_type': 'CRISPR-cas9',
  'perturbation_state': 1,
  'multi_perturbation': 1,
  'multi_sgRNA': 1},
 'TianKampmann2019_day7neuron': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 1,
  'multi_sgRNA': 1},
 'TianKampmann2019_iPSC': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 1,
  'multi_sgRNA': 1},
 'TianKampmann2021_CRISPRa': {'perturbation_type': 'CRISPRa',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
 'TianKampmann2021_CRISPRi': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1},
 'XuCao2023': {'perturbation_type': 'CRISPRi',
  'perturbation_state': 1,
  'multi_perturbation': 0,
  'multi_sgRNA': 1}}

prefix_list = list(gene_pert_dict.keys())

# GEARS都跑一遍

In [9]:
# 每个dataset用跑完的gears重新跑一遍

for prefix in list(gene_pert_dict.keys()):
    
    print('='*20, prefix)
    
    # - init para
    data_dir = '/nfs/public/lichen/data/single_cell/perturb_data/scPerturb/raw/scPerturb_rna/statistic_20240520'
    pert_cell_filter = 100 # this is used to filter perts, cell number less than this will be filtered
    seed = 2024 # this is the random seed
    split_type = 1 # 1 for unseen perts; 0 for unseen celltypes
    split_ratio = [0.7, 0.2, 0.1] # train:test:val; val is used to choose data, test is for final validation
    var_num = 5000 # selecting hvg number
    num_de_genes = 20 # number of de genes
    bs_train = 32 # batch size of trainloader
    bs_test = 32 # batch size of testloader
    lr = 1e-4

    tmp_dir = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark/scPerturb'
    # save_prefix = f'GEARS'
    save_prefix = f'GEARS_v2-prefix_{prefix}-pert_cell_filter_{pert_cell_filter}-\
seed_{seed}-split_type_{split_type}-var_num_{var_num}-num_de_genes_{num_de_genes}-bs_train_{bs_train}-\
bs_test_{bs_test}'

    save_dir = os.path.join(tmp_dir, prefix, save_prefix)

    # - load pert_data
    pert_data = pickle.load(open(os.path.join(save_dir,'pert_data.pkl'), 'rb'))

    # - get go genes; special set for GEARS
    pert_data.get_gene2go()
    pert_data.set_pert_genes()

    # - get dataset for gears
    pert_data.get_Data_gears(num_de_genes = pert_data.num_de_genes,
                            dataset_name = ['train', 'test', 'val'])

    # - add necessary elements for gears
    pert_data.modify_gears()

    # - get dataloader
    trainloader, testloader, valloader = pert_data.get_dataloader(mode='all')
    
    if len(testloader)==0:
        continue

    # - init gears model
    gears_model = GEARS(pert_data, device = 'cuda:3', 
                            weight_bias_track = False, 
                            proj_name = 'pertnet', 
                            exp_name = 'pertnet')
    gears_model.model_initialize(hidden_size = 64)
    
    # - load pretrain model
    tmp_dir = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark_202410/unseen_perturbation/method/GEARS_v1' # GEARS_v1: with control cells
    save_dir = os.path.join(tmp_dir, prefix)

    model_dir = os.path.join(save_dir,f'result_lr_0.0001')
    gears_model.load_pretrained(model_dir)
    
    # - Model testing
    test_loader = gears_model.dataloader['test_loader']
    test_res = evaluate(test_loader, gears_model.best_model,
                        gears_model.config['uncertainty'], gears_model.device)
    test_metrics, test_pert_res = compute_metrics(test_res)
    
    out = get_metric(pert_data.adata, test_res)
    # out = get_metric(pert_data.adata, test_res)
    # out_non_dropout = non_dropout_analysis(pert_data.adata, test_res)
    
    # - save the result
    tmp_dir = '/nfs/public/lichen/results/single_cell_perturbation/perturbation_benchmark_202410/unseen_perturbation/result/GEARS'
    save_prefix = 'GEARS_v2' 
    # GEARS_v1: the old version metric, with adding ctrl cells;  
    # GEARS_v2: the new version metric, with adding ctrl cells;  
    
    save_dir = os.path.join(tmp_dir, prefix, save_prefix)
    os.makedirs(save_dir, exist_ok=True)
    pickle.dump(out, open(os.path.join(save_dir,f'out.pkl'), 'wb'))

Found local copy...


==================== AdamsonWeissman2016_GSM2406675_10X001


Found local copy...
100%|██████████| 1/1 [00:00<00:00, 42.29it/s]


========== get Data_gears finished!
add adata finished
add condition finished
add set2conditions finished


Found local copy...


...

100%|██████████| 17/17 [00:00<00:00, 46.81it/s]


========== get Data_gears finished!
add adata finished
add condition finished
add set2conditions finished


Found local copy...


init changed
this is lichen version 2!
init changed
get metrics... ...


100%|██████████| 34/34 [00:18<00:00,  1.86it/s]
